# Join Statements - Lab

## Introduction

In this lab, you'll practice your knowledge of `JOIN` statements, using various types of joins and various methods for specifying the links between them.

## Objectives

You will be able to:
* Write SQL queries that make use of various types of joins
* Compare and contrast the various types of joins
* Discuss how primary and foreign keys are used in SQL
* Decide and perform whichever type of join is best for retrieving desired data

## CRM Schema

In almost all cases, rather than just working with a single table you will typically need data from multiple tables. 
Doing this requires the use of **joins** using shared columns from the two tables. 

In this lab, you'll use the same customer relationship management (CRM) database that you saw from the previous lesson.
<img src='images/Database-Schema.png' width="600">

## Connecting to the Database
Import the necessary packages and connect to the database `'data.sqlite'`.

In [2]:
import pandas as pd
import sqlite3
conn = sqlite3.Connection('data.sqlite')
cur = conn.cursor()

## Display the names of all the employees in Boston 

Hint: join the employees and offices tables.

In [7]:
cur.execute("""
SELECT *
FROM employees e
JOIN offices o
USING (officeCode)
WHERE o.city = 'Boston'
""")
df = pd.DataFrame(cur.fetchall())
df.columns = [x[0] for x in cur.description]
df

,employeeNumber,lastName,firstName,extension,email,officeCode,reportsTo,jobTitle,city,phone,addressLine1,addressLine2,state,country,postalCode,territory
0,1188,Firrelli,Julie,x2173,jfirrelli@classicmodelcars.com,2,1143,Sales Rep,Boston,+1 215 837 0825,1550 Court Place,Suite 102,MA,USA,02107,NA
1,1216,Patterson,Steve,x4334,spatterson@classicmodelcars.com,2,1143,Sales Rep,Boston,+1 215 837 0825,1550 Court Place,Suite 102,MA,USA,02107,NA


## Are there any offices that have zero employees?
Hint: Combine the employees and offices tables and use a group by.

In [15]:
cur.execute("""
SELECT o.officeCode, o.city, COUNT(e.employeeNumber) as num_employees
FROM offices o
LEFT JOIN employees e
USING (officeCode)
GROUP BY o.officeCode
HAVING COUNT(e.employeeNumber)=0
""")
cur.fetchall()
#no

[(27, 'Boston', 0)]

## Write 3 Questions of your own and answer them

In [28]:
#  Display the htmlDescription and employee's first and last name for each product that each employee has sold

cur.execute("""
SELECT e.firstName, e.lastName, pl.htmlDescription, p.productCode
FROM employees e
JOIN customers c
ON c.salesRepEmployeeNumber = e.employeeNumber
JOIN orders o
USING (customerNumber)
JOIN orderdetails od 
USING (orderNumber)
JOIN products p
USING (productCode)
JOIN productlines pl
USING (productLine)
WHERE pl.htmlDescription IS NOT NULL
""")
df = pd.DataFrame(cur.fetchall())
df.columns = [x[0] for x in cur.description]
df

,firstName,lastName,htmlDescription,productCode
0,Barry,Jones,,S10_1949
1,Barry,Jones,,S10_1949
2,Martin,Gerard,,S10_1949
3,Leslie,Jennings,,S10_1949
4,Mami,Nishi,,S10_1949
...,...,...,...,...
2991,Gerard,Hernandez,,S50_1341
2992,Steve,Patterson,,S50_1341
2993,Loui,Bondur,,S50_1341
2994,George,Vanauf,,S50_1341


In [21]:
# Top Selling Employee
cur.execute("""
SELECT e.employeeNumber, e.firstName, e.lastName, COUNT(p.productCode)
FROM employees e
JOIN customers c
ON c.salesRepEmployeeNumber = e.employeeNumber
JOIN orders o
USING (customerNumber)
JOIN orderdetails od 
USING (orderNumber)
JOIN products p
USING (productCode)
GROUP BY e.employeeNumber
ORDER BY COUNT(p.productCode) DESC
LIMIT 1;
""")
df = pd.DataFrame(cur.fetchall())
df.columns = [x[0] for x in cur.description]
df

,employeeNumber,firstName,lastName,COUNT(p.productCode)
0,1370,Gerard,Hernandez,396


In [23]:
# How many customers have outstanding orders?
cur.execute("""
SELECT COUNT(c.customerNumber), o.status
FROM customers c
JOIN orders o
USING (customerNumber)
GROUP BY o.status

""")
df = pd.DataFrame(cur.fetchall())
df.columns = [x[0] for x in cur.description]
df

,COUNT(c.customerNumber),status
0,6,Cancelled
1,3,Disputed
2,6,In Process
3,4,On Hold
4,4,Resolved
5,303,Shipped


## Level Up: Display the names of every individual product that each employee has sold

In [24]:
cur.execute("""
SELECT e.firstName, e.lastName, p.productCode
FROM employees e
JOIN customers c
ON c.salesRepEmployeeNumber = e.employeeNumber
JOIN orders o
USING (customerNumber)
JOIN orderdetails od 
USING (orderNumber)
JOIN products p
USING (productCode)
""")
df = pd.DataFrame(cur.fetchall())
df.columns = [x[0] for x in cur.description]
df

,firstName,lastName,productCode
0,Leslie,Jennings,S12_1666
1,Leslie,Jennings,S18_1097
2,Leslie,Jennings,S18_4668
3,Leslie,Jennings,S32_3522
4,Leslie,Jennings,S12_1099
...,...,...,...
2991,Martin,Gerard,S32_2509
2992,Martin,Gerard,S32_3207
2993,Martin,Gerard,S50_1392
2994,Martin,Gerard,S18_2248


## Level Up: Display the Number of Products each employee has sold

In [29]:
cur.execute("""
SELECT e.employeeNumber, e.firstName, e.lastName, COUNT(p.productCode)
FROM employees e
JOIN customers c
ON c.salesRepEmployeeNumber = e.employeeNumber
JOIN orders o
USING (customerNumber)
JOIN orderdetails od 
USING (orderNumber)
JOIN products p
USING (productCode)
GROUP BY e.employeeNumber
""")
df = pd.DataFrame(cur.fetchall())
df.columns = [x[0] for x in cur.description]
df

,employeeNumber,firstName,lastName,COUNT(p.productCode)
0,1165,Leslie,Jennings,331
1,1166,Leslie,Thompson,114
2,1188,Julie,Firrelli,124
3,1216,Steve,Patterson,152
4,1286,Foon Yue,Tseng,142
5,1323,George,Vanauf,211
6,1337,Loui,Bondur,177
7,1370,Gerard,Hernandez,396
8,1401,Pamela,Castillo,272
9,1501,Larry,Bott,236


## Summary

Congrats! You practiced using join statements and leveraged your foreign keys knowledge!